# Azure AI Agents na may Suporta para sa Model Context Protocol (MCP)

Ipinapakita ng notebook na ito kung paano gamitin ang Azure AI Agents na may mga tool ng Model Context Protocol (MCP) upang makagawa ng isang matalinong ahente na maaaring gumamit ng mga panlabas na MCP server para sa mas pinahusay na kakayahan.


## I-install ang Kinakailangang NuGet Packages

Una, kailangan nating i-install ang Azure AI Agents Persistent package na nagbibigay ng pangunahing kakayahan para sa paggamit ng Azure AI Agents.


## Mga Benepisyo ng Keyless Authentication

Ipinapakita ng notebook na ito ang **keyless authentication** na nagbibigay ng ilang mga benepisyo:
- ✅ **Walang API keys na kailangang pamahalaan** - Gumagamit ng Azure identity-based authentication
- ✅ **Mas pinahusay na seguridad** - Walang mga lihim na nakaimbak sa code o configuration
- ✅ **Awtomatikong pag-ikot ng kredensyal** - Ang Azure ang humahawak sa lifecycle ng kredensyal
- ✅ **Access na nakabatay sa role** - Gumagamit ng Azure RBAC para sa mas detalyadong mga pahintulot

Ang `DefaultAzureCredential` ay awtomatikong gagamit ng pinakamahusay na magagamit na pinagmulan ng kredensyal:
1. Managed Identity (kapag tumatakbo sa Azure)
2. Mga kredensyal ng Azure CLI (sa panahon ng development)
3. Mga kredensyal ng Visual Studio
4. Mga environment variable (kung naka-configure)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

I-install ang Azure Identity package para sa authentication sa mga Azure services gamit ang DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## I-import ang Kinakailangang Mga Namespace

I-import ang mga kinakailangang namespace para sa Azure AI Agents at Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## I-configure ang Azure AI Agent Client (Keyless Authentication)

I-set up ang mga configuration variable at gumawa ng PersistentAgentsClient gamit ang **keyless authentication**:
- **projectEndpoint**: Ang endpoint ng proyekto sa Azure AI Foundry
- **modelDeploymentName**: Ang pangalan ng na-deploy na AI model (GPT-4.1 nano)
- **mcpServerUrl**: Ang URL ng MCP server (Microsoft Learn API)
- **mcpServerLabel**: Isang label para tukuyin ang MCP server
- **DefaultAzureCredential**: Gumagamit ng managed identity, Azure CLI, o iba pang pinagmulan ng kredensyal (hindi kinakailangan ang API keys)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Gumawa ng MCP Tool Definition

Gumawa ng MCP tool definition na kumokonekta sa Microsoft Learn MCP server. Magbibigay-daan ito sa ahente na ma-access ang nilalaman at dokumentasyon ng Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Gumawa ng AI Agent

Gumawa ng permanenteng AI agent gamit ang tinukoy na modelo at mga MCP tools. Ang agent ay naka-configure na may:
- Ang GPT-4.1 nano model
- Mga tagubilin para gamitin ang MCP tools bilang tulong
- Access sa Microsoft Learn MCP server


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Gumawa ng Thread at Magpadala ng Mensahe

Gumawa ng thread ng pag-uusap at magpadala ng mensahe sa user na nagtatanong tungkol sa pagkakaiba ng Azure OpenAI at OpenAI. Susubukan nito ang kakayahan ng ahente na gamitin ang MCP tools upang magbigay ng tamang impormasyon.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## I-configure ang Mga Resource ng MCP Tool (Walang Susi)

I-set up ang mga resource ng MCP tool. Para sa isang tunay na walang susi na paraan, maaari mong alisin ang mga custom na header kung sinusuportahan ng MCP server ang Azure identity-based authentication. Ang halimbawa sa ibaba ay nagpapakita kung paano magdagdag ng mga header kung kinakailangan, ngunit para sa mga walang susi na sitwasyon, maaaring hindi na ito kailangan.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Simulan ang Pagpapatakbo ng Ahente

Gumawa at simulan ang pagpapatakbo upang iproseso ang mensahe ng gumagamit. Gagamitin ng ahente ang nakatakdang MCP na mga tool at mapagkukunan upang makabuo ng sagot.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Subaybayan ang Takbo at Pangasiwaan ang Mga Pag-apruba ng Tool (Keyless)

Subaybayan ang status ng takbo ng agent at pangasiwaan ang anumang kinakailangang pag-apruba ng tool. Ang loop na ito:
1. Naghihintay na matapos ang takbo o mangailangan ng aksyon
2. Awtomatikong inaaprubahan ang mga tawag sa MCP tool kapag kinakailangan
3. Para sa keyless authentication, maaaring hindi kailanganin ang mga header kung sinusuportahan ng MCP server ang Azure identity


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Ipakita ang Resulta ng Usapan

Kunin at ipakita ang lahat ng mensahe sa thread, kabilang ang tanong ng user at sagot ng ahente. Ang mga mensahe ay ipinapakita ayon sa pagkakasunod-sunod ng oras na may mga tag ng oras at indikasyon ng papel.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
